# import Libraries

In [ ]:
import pandas as pd
import seaborn as sns

# read dataset 

In [ ]:
data_df = pd.read_csv('/kaggle/input/pima-indians-diabetes-database/diabetes.csv',na_values={'Glucose':0,'BloodPressure':0,'SkinThickness':0,'Insulin':0,'BMI':0})

## explore dataframe characteristics

In [ ]:
data_df.head()

In [ ]:
data_df.info()

In [ ]:
data_df.describe().T

### we notice lots of null values

## explore data

In [ ]:
sns.pairplot(data_df,hue='Outcome')

In [ ]:
correlation_matrix = data_df.corr()
sns.heatmap(correlation_matrix, cmap='Reds',annot=True)

### get columns with null

In [ ]:
isna_c = data_df.isna().sum()
isna_c[isna_c > 0]

In [ ]:
data_df.groupby('Outcome').median().T

## fix Clucose data

In [ ]:
sns.displot(data_df.Glucose)

In [ ]:
data_df.Glucose.fillna(data_df.groupby('Outcome')['Glucose'].transform('median'),inplace=True)

In [ ]:
sns.displot(data_df.Glucose)

In [ ]:
sns.pairplot(data_df[['Outcome','Glucose']],hue='Outcome')

## fix BloodPressure

In [ ]:
data_df['bmi_cat'] = data_df.BMI.apply(lambda x: 'U' if x < 18.5 else 'N' if x < 25 else 'V' if x < 30 else 'O')
data_df['age_cat'] = data_df.Age.apply(lambda x: 'A' if x < 30 else 'B' if x < 40 else 'C' if x < 50 else 'D')
data_df.bmi_cat = data_df.bmi_cat.astype('category')
data_df.age_cat = data_df.age_cat.astype('category')

In [ ]:
sns.pairplot(data_df[['bmi_cat','BloodPressure','age_cat']],hue='bmi_cat')

In [ ]:
data_df[['bmi_cat','BloodPressure','age_cat']].groupby(['bmi_cat','age_cat']).median()

In [ ]:
data_df.BloodPressure.fillna(data_df.groupby(['bmi_cat','age_cat'])['BloodPressure'].transform('median'),inplace=True)

In [ ]:
sns.pairplot(data_df[['Outcome','BloodPressure']],hue='Outcome')

In [ ]:
isna_c = data_df.isna().sum()
isna_c[isna_c > 0]

In [ ]:
sns.boxplot(data=data_df[['bmi_cat','BloodPressure']],y='BloodPressure',x='bmi_cat')

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(data=data_df[['Age','BloodPressure']],y='BloodPressure',x='Age')

In [ ]:
data_df.BMI.fillna(data_df.groupby(['Outcome','age_cat'])['BMI'].transform('median'),inplace=True)

In [ ]:
sns.pairplot(data_df[['Outcome','BMI']],hue='Outcome')

In [ ]:
isna_c = data_df.isna().sum()
isna_c[isna_c > 0]

## exploring skinthikness

In [ ]:
data_df.SkinThickness.fillna(data_df.groupby(['Outcome','bmi_cat'])['SkinThickness'].transform('median'),inplace=True)

In [ ]:
sns.pairplot(data_df[['Outcome','SkinThickness']],hue='Outcome')

## Insulin preparation

In [ ]:
sns.pairplot(data_df[['Outcome','Insulin']],hue='Outcome')

In [ ]:
sns.boxplot(data=data_df[['Outcome','Glucose']],x='Outcome',y='Glucose')

In [ ]:
data_df['glu_cat'] = data_df.Glucose.apply(lambda x: 'N' if x < 140 else 'U')


In [ ]:
data_df.Insulin.fillna(data_df.groupby(['bmi_cat','glu_cat','Outcome'])['Insulin'].transform('median'),inplace=True)

In [ ]:
sns.pairplot(data_df[['Outcome','Insulin']],hue='Outcome')

In [ ]:
sns.pairplot(data_df,hue='Outcome')

# modeling

In [ ]:
data_df.info()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data_df.drop(['Outcome','glu_cat','age_cat','bmi_cat'],axis=1), data_df.Outcome, test_size=0.20, random_state=42)

## using KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=7)
neigh.fit(X_train, y_train)
neigh.score(X_train,y_train)

In [ ]:
neigh.score(X_test,y_test)

In [ ]:
from sklearn import metrics
y_pred = neigh.predict(X_test)
cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
cnf_matrix

In [ ]:
sns.heatmap(cnf_matrix)

## using logistic regression

In [ ]:
X_train.describe().T

## Error with Logistic Regression

In [ ]:
# from sklearn.linear_model import LogisticRegression
# clf = LogisticRegression(random_state=42)
# clf.fit(X_train, y_train)
# clf.score(X_train,y_train)

In [ ]:
# clf.score(X_test,y_test)

In [ ]:
# y_pred = clf.predict(X_test)
# cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
# cnf_matrix

In [ ]:
# sns.heatmap(cnf_matrix)

## SVM

In [ ]:
from sklearn import svm
clf2 = svm.SVC()
clf2.fit(X_train, y_train)
clf2.score(X_train, y_train)

In [ ]:
clf2.score(X_test,y_test)

In [ ]:
y_pred = clf2.predict(X_test)
cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
cnf_matrix

In [ ]:
sns.heatmap(cnf_matrix)

In [ ]:
from sklearn.model_selection import cross_validate

In [ ]:
 cross_validate(neigh, data_df.drop(['Outcome','glu_cat','age_cat','bmi_cat'],axis=1), data_df.Outcome, cv=3)

In [ ]:
 cross_validate(clf2, data_df.drop(['Outcome','glu_cat','age_cat','bmi_cat'],axis=1), data_df.Outcome, cv=3)